In [2]:
import numpy as np 
import scipy as sp

import control as ct
from core.utility import Output_Con

In [12]:
A = np.eye(2)
B = np.array([[0],[1]])
C = np.eye(2)

In [13]:
dt = 0.1

In [14]:
Ad = A*dt + np.eye(A.shape[0])
Bd = B*dt
Cd  = C

In [15]:
Con = ct.ctrb(A,B)
print("Rank for state Controllability :", np.linalg.matrix_rank(Con))

Out_con, Out_rank = Output_Con(A, B, C)
print("Rank for Output Controllability :", Out_rank)


Rank for state Controllability : 1
Rank for Output Controllability : 1


In [16]:
A

array([[1., 0.],
       [0., 1.]])

In [17]:
B

array([[0],
       [1]])